# Tuning a Semantic Segmentation Transformers

We have now learned the power of the transformer architecture.
One of its applications is to perform semantic segmentation---
a computer vision task for associating each pixel in an image into a class or object.

Althgouh training a full transformer from scratch can be expensive, it is relatively easy to "tune" a pre-trained transformer to different types of data.
In this hands-on, we will use an [`implementation`](https://github.com/huggingface/transformers) of [SegFormer](https://arxiv.org/pdf/2105.15203.pdf) by [hugging face](https://huggingface.co/).

## Pre-trained Model

To run this hands-on with reasonable resources, we will use the [`MiT-b0`](https://huggingface.co/nvidia/mit-b0) variant of the SegFormer, 
which has only [3.7M parameters](https://huggingface.co/docs/transformers/en/model_doc/segformer).
Following the documentation, we can use it by the following steps.

**Step 1a.** Load the necessary python packages.

In [1]:
import requests       # for download image
from PIL import Image # for reading image
from transformers import SegformerImageProcessor, SegformerForImageClassification

**Step 1b.** Download a sample image and read it as a Pillow image.

In [2]:
url = "http://images.cocodataset.org/val2017/000000039769.jpg"
img = Image.open(requests.get(url, stream=True).raw)

**Step 1c.** Download the pre-trained model and construct the image processor and classifier.

In [3]:
impro = SegformerImageProcessor.from_pretrained("nvidia/mit-b0")
model = SegformerForImageClassification.from_pretrained("nvidia/mit-b0")

/Users/ckc/.venv/dl/lib/python3.12/site-packages/transformers/models/segformer/image_processing_segformer.py:102: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


**Step 1d.** Process the input image and run it through the model. 

In [4]:
inputs  = impro(images=img, return_tensors="pt")
outputs = model(**inputs)

**Step 1e.** Obtain and print the predicted class---the model predicts one of the 1000 ImageNet classes

In [5]:
logits  = outputs.logits
predicted_idx = logits.argmax(-1).item()
print("Predicted class:", model.config.id2label[predicted_idx])

Predicted class: tabby, tabby cat
